In [1]:
from sklearn.datasets import make_classification
import math
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import warnings
warnings.filterwarnings("ignore")

## generate data with 4 features and 4 classes

In [2]:
#generate data 
# change the value of n_samples to change the number of inputs
X, y = make_classification(n_samples=5000, n_features=4, n_informative=4, n_redundant=0, n_classes=4)

## change below parameters to your liking for the neural network along with some restrictions as defined below

In [3]:
base_params = {
    "hidden_layers": 1,        # can change to any value
    "num_neurons_hidden" : 4,  # only 4 acceptable
    "num_neurons_output" : 4,  # only 4 acceptable
    "activation_func": "sigmoid", # use tanh or sigmoid
    "learning_rate" : 0.35,          # must be between 0 and 1
    "train_test-split-ratio": 0.80, # can be any ratio. 0.80 here means that 80% of the data will be used as training set
    "mini_batch_size" : 500,  
    "epochs" : 5
    
}

## Below cell will define the neural network function

In [4]:
# feedforward
def neural_network(base_params, training_sample, testing_sample):

# calculate layers
    layers = [len(training_sample[0][0])] + [base_params["num_neurons_hidden"]]*base_params["hidden_layers"] + [base_params["num_neurons_output"]]
    
    ## initialize weights & biases
    weights = []
    for i in range(len(layers)):
        j = i+1
        if j < len(layers):
            weights.append(np.array(np.random.default_rng().standard_normal(layers[i]*layers[j])).reshape(layers[i],layers[i]))
    weights = np.array(weights)
    biases = np.array([np.random.default_rng().standard_normal(i) for i in layers[1:]])
    
    # for each epoch make mini batches
    for epoch in range(base_params["epochs"]):
        np.random.shuffle(training_sample)
        for batch in range(0,len(training_sample), base_params["mini_batch_size"]):
            training_sample[batch:batch + base_params["mini_batch_size"]]
            for x_values in range(len(training_sample)):
                all_activations = feedforward(layers,weights, biases, training_sample[x_values][0], training_sample[x_values][1])
                # take total deltas 
                if epoch == 0:
                    all_deltas = backpropagate_error(layers,weights,all_activations, training_sample[x_values][1])
                else:
                    all_deltas = all_deltas + backpropagate_error(layers,weights,all_activations, training_sample[x_values][1])
            weights, biases  = update_weights_biases(weights, biases, all_deltas, all_activations)
        print("=========================================")
        print( "epochs : " + str(epoch + 1) + " / " + str(base_params["epochs"]))
        calc_accuracy(layers, testing_sample,weights, biases)
        print("====================================")
 
def feedforward(layers,weights, biases, train_x, train_y):
    all_activations = []
    with_activation = train_x
    all_activations.append(with_activation)
    for layer in range(1,len(layers)):
        all_activations.append( activation_func(np.dot(weights[layer-1],with_activation) + biases[layer-1], base_params["activation_func"]))
        with_activation=all_activations[layer-1]
    all_activations = np.array(all_activations)
    return all_activations

def backpropagate_error(layers,weights, all_activations, train_y):
    # last laeyer delta
    deltas = []
    deltas.append( np.multiply( derivative_of_cost(all_activations[-1],train_y), derivate_act_func(all_activations[-1], base_params["activation_func"])))

    # all layers delta
    for delta_layer in range(len(weights),1, -1):

        one_delta = np.multiply(np.dot(np.transpose(weights[delta_layer - 1]),deltas[len(weights) - delta_layer]), derivate_act_func(all_activations[delta_layer - 1],base_params["activation_func"]))
        deltas.append(one_delta)

    # 0 index of deltas represent last layer delta
    deltas = np.flipud(np.array(deltas))
    return deltas

def update_weights_biases(weights, biases, deltas, all_activations):
    # updation of weights and biases
    for i in range(len(weights)):
        for j in range(len(weights[i])):
            delta_weights = np.transpose(weights[i])
            delta_weights[i][j] = delta_weights[i][j] - ( base_params["learning_rate"] / base_params["mini_batch_size"]) * (all_activations[i][j]*deltas[i][j])
        weights[i] = np.transpose(delta_weights)
    biases = biases - (base_params["learning_rate"]/ base_params["mini_batch_size"])*deltas
    return weights, biases

def calc_accuracy(layers,sample_to_check, weights, biases):
    predicted_outputs = []
    actual_outputs = []
    for i in range(len(sample_to_check)):
        predicted_outputs.append(np.argmax(feedforward(layers,weights, biases, sample_to_check[i][0], sample_to_check[i][1] )[-1]))
        actual_outputs.append(np.argmax(sample_to_check[i][1]))
    print(" precision = " + str(precision_score(predicted_outputs, actual_outputs, average='macro')))
    print(" recall = " + str(recall_score(predicted_outputs, actual_outputs, average='macro')))
    print(" accuracy = " + str(accuracy_score(predicted_outputs, actual_outputs)))

def activation_func(z, func_to_use):
    if func_to_use == "sigmoid":
        return sigmoid_func(z)
    else:
        return tanh_func(z)

def sigmoid_func(z):
    return 1/(1+math.e**(-z))

def tanh_func(z):
    return ((math.e**(z)) - (math.e**(-z)))/ ((math.e**(z)) + (math.e**(-z)) )

def derivate_act_func(output, func_to_use):
    if func_to_use == "sigmoid":
        return derivative_of_sigmoid(output)
    else:
        return derivative_of_tanh(output)

# error at output layer
def derivative_of_cost(output,target):
    return output - target

def derivative_of_sigmoid(output):
    return output*(1-output)

def derivative_of_tanh(output):
    return 1 - output**2



# Run below cell to train your neural network and output accuracy, precision, recall

In [5]:
##data_preprocessing
input_data_len = X.shape[0]
y_classes = np.unique(y)
size_train = int(base_params["train_test-split-ratio"]*input_data_len)
train_x, train_y , test_x, test_y = X[:size_train], y[:size_train], X[size_train:], y[size_train:]

zero_vector = np.zeros((size_train, len(y_classes)))
for i in range(len(train_y)):
    zero_vector[i][train_y[i]] = 1
# replace train y with vectors. the index have 1 represents the correct value
train_y = zero_vector
training_sample = np.array(list(zip(train_x, train_y)))
testing_sample = np.array(list(zip(test_x, test_y)))


neural_network(base_params, training_sample, testing_sample)

epochs : 1 / 5
 precision = 0.05025
 recall = 0.25
 accuracy = 0.201
epochs : 2 / 5
 precision = 0.038
 recall = 0.25
 accuracy = 0.152
epochs : 3 / 5
 precision = 0.03075
 recall = 0.25
 accuracy = 0.123
epochs : 4 / 5
 precision = 0.0135
 recall = 0.25
 accuracy = 0.054
epochs : 5 / 5
 precision = 0.0026666666666666666
 recall = 0.3333333333333333
 accuracy = 0.008
